## Методы обработки естественных языков

### Практическая работа 1

Завимости необходимые для запуска программы:
- [spacy](https://spacy.io/)
    - `spacy download ru_core_news_sm`
- [ruwordnet](https://github.com/avidale/python-ruwordnet)
    - `ruwordnet download`
- [scikit-learn](https://scikit-learn.org/stable/)

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import spacy
import pandas as pd

from ruwordnet import RuWordNet
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

Документ - роман Ф.М.Достоевского "Преступление и наказание"

In [3]:
book_text = ""
with open("data/Преступление_и_наказание.txt", "r") as f:
    book_text = f.read()

In [4]:
print(book_text[:2000])

[]

Достоевский Ф.М.

Преступление и наказание / Достоевский Ф.М.—М.:ДА!Медиа, 2014.

«Преступление и наказание» – роман великого русского писателя,
мыслителя, философа и публициста Федора Михайловича Достоевского
(1821-1881). Написан в 1865-1866 годы. Впервые напечатан по частям в
январе-декабре 1866 года в журнале «Русский вестник». Вскоре после
окончания журнальной публикации романа Достоевский печатает его
отдельным изданием, для которого сделал в тексте значительные сокращения
и изменения: три части журнальной редакции были преобразованы в шесть,
частично было изменено и деление на главы.

«Преступление и наказание» открывает новый, высший этап творчества
Достоевского. Здесь он впервые выступает как создатель принципиально
нового романа в мировой литературе, который был назван «полифоническим».
В основе драматического конфликта произведения – столкновение
характеров, воплощающих разные идейные принципы, борьба теории с жизнью
в душе каждого одержимого человека. Изображение обществ

In [5]:
nlp = spacy.load("ru_core_news_sm")
wn = RuWordNet()

#### Нормализация

- для лемматизации используется библиотека `spaCy`
- для синонимизации используется библиотека `ruwordnet`

In [6]:
def normalize_text(text):
    doc = nlp(text)

    # лемматизация и удаление стоп-слов
    lemmas = [token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop]

    # синонимизация
    tokens = []
    for token in lemmas:
        senses = wn.get_senses(token)
        synonym = None
        if len(senses) != 0:
            synonym = wn.get_senses(token)[0].synset.senses[0].name.lower()
        if synonym:
            tokens.append(synonym)
        else:
            tokens.append(token)

    return tokens

Так как синонимизация занимает очень много времени, из-за этого используется не вся книга

In [7]:
normalized_tokens = normalize_text(book_text[:50_000])

#### Анализатор тональности

[Russian Sentiment Dataset](https://www.kaggle.com/datasets/mar1mba/russian-sentiment-dataset) - датасет с размеченными данными для анализа тональности.

Структура набора данных:
- `text` - текст отзыва или комментария
- `label` - метка настроения:
    - `0` - нейтральный
    - `1` - положительный
    - `2` - отрицательный
- `src` - источник набора данных

In [8]:
data = pd.read_csv("data/sentiment_dataset.csv")

texts = data["text"].values
labels = data["label"].values

In [9]:
x_train, x_test, y_train, y_test = train_test_split(texts, labels, random_state=0)

##### Векторизация

In [10]:
vectorizer = CountVectorizer()
x_train_vec = vectorizer.fit_transform(x_train)
x_test_vec = vectorizer.transform(x_test)

##### Обучение наивного байесовского классификатора

In [11]:
clf = MultinomialNB()
clf.fit(x_train_vec, y_train);

In [12]:
y_pred = clf.predict(x_test_vec)

print(f"Оценка точности классификтора: {accuracy_score(y_test, y_pred):.2f}")

Оценка точности классификтора: 0.68


In [13]:
normalized_book = " ".join(normalized_tokens)
vectorized_book = vectorizer.transform([normalized_book])
prection_book = clf.predict(vectorized_book)[0]

#### Итоги практической работы

In [14]:
labels_dict = {0: "нейтральную", 1: "положительную", 2: "отрицательную"}

print(f"Часть книги Ф.М.Достоевского 'Преступление и наказание' имеет {labels_dict[prection_book]} тональность")

Часть книги Ф.М.Достоевского 'Преступление и наказание' имеет нейтральную тональность
